In [13]:
class MyMonad m where
    (>>==) :: m a -> (a -> m b) -> m b
    return :: a -> m a

class MyApplicative f where
    pure :: a -> f a
    (<**>) :: f (a -> b) -> f a -> f b

class MyFunctor f where
    fmap :: (a -> b) -> f a -> f b
    

In [14]:
-- instance Monoid FList a where


-- instance Semigroup (FList a) where
class MyFunctor f where
    fmap :: (a->b) -> f a -> f b

class MyMonoid m where
    mempty :: m
    mappend :: m -> m -> m

instance MyFunctor Maybe where
    fmap :: (a->b) -> Maybe a -> Maybe b
    fmap f (Just a) = Just (f a)
    fmap f _ = Nothing

instance MyFunctor [] where
    fmap :: (a -> b) -> [a] -> [b]
    fmap f (x:xs) = f x : fmap f xs
    fmap f _ = []


instance MyMonoid [a] where
    mempty = []
    mappend = (++)

instance MyMonoid Int where
    mempty = 0
    mappend = (+)

-- data Maybe a = Just a | Nothing
-- :i Monoid

In [80]:
newtype Env a b = Env (a -> Maybe b)


instance MyFunctor (Env a) where
    fmap :: (b -> c) -> Env a b -> Env a c
    fmap f (Env g) = Env (\c -> case g c of 
                                    Nothing -> Nothing
                                    Just b -> Just $ f b)


instance MyApplicative (Env a) where
    pure :: b -> Env a b
    pure b = Env (\_ -> Just b)

    (<**>) :: Env a (b -> c) -> Env a b -> Env a c
    Env f <**> Env g = Env (\x -> case g x of
                                Just y -> case f x of 
                                    Just h -> Just $ h y
                                    _ -> Nothing
                                _ -> Nothing)


instance MyMonad (Env r) where
    return = pure
    (>>==) :: Env r a -> (a -> Env r b) -> Env r b
    Env g >>== f = Env (\x -> case g x of 
                                    Nothing -> Nothing
                                    Just z -> e x where Env e = f z)